# 00 — Data Ingestion (Step-by-step)
**Goal:** Walk through using `get_market_data` and `get_options_data` with caching.

---


In [3]:
# 1. Environment setup
import os
from datetime import date
from oami.config import initialize_environment
from oami.data_layer import get_market_data, get_options_data

api_key = initialize_environment()
print('API key configured' if api_key != 'YOUR_KEY_HERE' else 'Running in offline mode using cache')


In [ ]:
## 2. Choose symbol and date window
We'll work with `SPY` and fetch the last 30 days of data.


In [ ]:
from datetime import timedelta
SYMBOL = 'SPY'
end_date = date.today()
start_date = end_date - timedelta(days=30)
print(f'Requesting data from {start_date} to {end_date}')


In [ ]:
## 3. Fetch market data
`get_market_data` will re-use cached CSVs in `data/csv/market/<interval>/<symbol>.csv`. Missing ranges are pulled from Polygon and persisted automatically.

In [ ]:
market_df = get_market_data(
    symbol=SYMBOL,
    start=start_date.isoformat(),
    end=end_date.isoformat(),
    interval='1D',
    use_cache=True,
)
print('Market rows:', len(market_df))
market_df.head()


In [ ]:
## 4. Fetch options data
`get_options_data` pulls contract metadata, stores per-contract aggregates in `data/options`, then builds a daily sentiment summary.

## 5. Where was data saved?
- Market cache: `data/csv/market/1D/{SYMBOL}.csv`
- Contracts: `data/options/{SYMBOL}/contracts/<YYYY-MM-DD>.csv`
- Aggregates per contract: `data/options/{SYMBOL}/1day/<expiry>/<contract>.csv`
- Options summary: `data/csv/options/1D/{SYMBOL}.csv`

Subsequent calls within the same window load directly from these caches.